In [ ]:
#run script to extract data from pcap
#in this script, input data format is:
#time, src, dst, length, protocol

In [3]:
#spark
__author__ = 'vcoder'

import os
import sys

#change them when spark and pyspark path changes

#SPARK_HOME = "/home/vcoder/EDA/spark-1.5.0"
#SPARK_HOME_PYTHON = SPARK_HOME + "/python"

SPARK_HOME = "/home/worker/software/spark-1.5.0"
SPARK_HOME_PYTHON = SPARK_HOME + "/python"

os.environ['SPARK_HOME'] = SPARK_HOME
sys.path.append(SPARK_HOME_PYTHON)

from pyspark import SparkContext
from pyspark import SparkConf

import geoip2.database
reader = geoip2.database.Reader('GeoLite2-City.mmdb')

def ip2city(ip):
    try:
        city = reader.city(ip).city.name
    except:
        city = 'not found'
    return city

def ip2la(ip):
    try:
        la = reader.city(ip).location.latitude
    except:
        la = 0.
    return la

def ip2lo(ip):
    try:
        lo = reader.city(ip).location.longitude
    except:
        lo = 0.
    return lo

def ip2country(ip):
    try:
        co = reader.city(ip).country.name
    except:
        co = 'not found'
    return co


In [5]:
sc = SparkContext('local', 'ip2geo')

In [3]:
csvPath = 'csv/output.csv'

In [ ]:
import sys
#number of top x
x = int(sys.argv[2])

In [25]:
raw = (sc.textFile(csvPath).map(lambda x: x.split(','))
       .map(lambda x: (x[1], x[2], float(x[3]), x[4]))) #delimiter may need to change
                      #src, dst, dataLength, protocol

# src_topXpkts = (raw.map(lambda x: (x[0], x[1])).groupByKey()
#        .map(lambda x: (x[0], len(x)-1))).takeOrdered(x, key = lambda x: -x[1])

# dst_topXpkts = (raw.map(lambda x: (x[1], x[0])).groupByKey()
#        .map(lambda x: (x[0], len(x)-1))).takeOrdered(x, key = lambda x: -x[1])

# src_topXdata = (raw.map(lambda x: (x[0], x[2])).groupByKey()
#        .map(lambda x: (x[0], sum(x[1:])))).takeOrdered(x, key = lambda x: -x[1])

# dst_topXdata = (raw.map(lambda x: (x[1], x[2])).groupByKey()
#        .map(lambda x: (x[0], sum(x[1:])))).takeOrdered(x, key = lambda x: -x[1])

# protocol_topX = raw.map(lambda x: (x[3], 1)).groupByKey().map(lambda x:(x[0], sum(x[1:])))
# num = protocol_topX.count()
# if nums > x:
#     num = x
# prtcol_topX = protocol_topX.takeOrdered(num, key = lambda x: -x[1])

[u'198.241.159.205', u'60.50.181.52', u'188.72.200.132', u'64.113.118.122', u'41.215.236.152', u'202.1.175.252', u'195.198.120.238', u'71.126.222.64', u'198.241.158.183']


In [ ]:
src_topXpkts = (raw.map(lambda x: (x[0], x[1])).groupByKey()
       .map(lambda x: {'ip':x[0], 'rate':len(x)-1})).takeOrdered(x, key = lambda x: -x['rate'])

dst_topXpkts = (raw.map(lambda x: (x[1], x[0])).groupByKey()
       .map(lambda x: {'ip':x[0], 'rate':len(x)-1})).takeOrdered(x, key = lambda x: -x['rate'])

src_topXdata = (raw.map(lambda x: (x[0], x[2])).groupByKey()
       .map(lambda x: {'ip':x[0], 'rate':sum(x[1:])}).takeOrdered(x, key = lambda x: -x['rate'])

dst_topXdata = (raw.map(lambda x: (x[1], x[2])).groupByKey()
       .map(lambda x: {'ip':x[0], 'rate':sum(x[1:])}).takeOrdered(x, key = lambda x: -x['rate'])


In [ ]:
protocol_topX = raw.map(lambda x: (x[3], 1)).groupByKey().map(lambda x:{x[0], sum(x[1:])})
num = protocol_topX.count()
if nums > x:
    num = x
prtcol_topX = protocol_topX.takeOrdered(num, key = lambda x: -x.values[0])

IP = []
for x in prorol_topX:
    IP.append({
            'country' : x.keys[0],
            'rate' : x.values[0]
        })

TopCountrySchema = {
    'recordID' : 1, 
    'IP' : IP
}


In [ ]:
TopSrcIPpktsSchema = {
    'IP' : src_topXpkts
}
TopDstIPpktsSchema = {
    'IP' : dst_topXpkts
}

TopSrcIPdataSchema = {
    'IP' : src_topXdata
}
TopDstIPdataSchema = {
    'IP' : dst_topXdata
}

In [35]:
import pymongo
from pymongo import MongoClient
client = MongoClient()

dbName = 'test'
collectionName = 'Maps'

db = client[dbName]
collection = db[collectionName]

collection.insert_one(ipDistribution)

In [36]:
sc.stop()

In [4]:
b = [1,2,3,4,5]
c = sc.parallelize(b)
print b.collect()

NameError: name 'sc' is not defined